In [243]:
# Initial imports
import os
import re
import json
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pandas import json_normalize
from dotenv import load_dotenv
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, CategoriesOptions, ConceptsOptions, EmotionOptions, EntitiesOptions, KeywordsOptions, RelationsOptions, SemanticRolesOptions
from dotenv import load_dotenv
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
load_dotenv()

True

In [209]:
# Connect to IBM API
nlu_apikey = "insert here"
nlu_url = "insert here"

In [210]:
# Import Dataset 
dataset = pd.read_csv('/Users/erahm/chatter/front-end/src/CoralReef.csv')
#dataset.columns = ["Message"]
dataset.head(100)

,Unnamed: 0,0
0,0,RT @IndoPacScarini: Did u know cyanobacteria u...


Data Cleaning

In [211]:
# Drop any duplicate messages
updated_df = dataset.drop_duplicates(keep='last')
updated_df

,Unnamed: 0,0
0,0,RT @IndoPacScarini: Did u know cyanobacteria u...


In [ ]:
# Print dataset as string
dataset_string = updated_df.to_string()
dataset_string

In [ ]:
    # Data Cleaning
    
    # Convert everything to lowercase
    dataset_string = dataset_string.lower() 
    
    # Remove mentions   
    dataset_string = re.sub('@','', dataset_string)  
    
    # Remove urls
    #dataset_string = re.sub(r'https?:\/\/.*\/\w*', '', dataset_string)
    
    # Remove hashtags
    dataset_string = re.sub(r'#\w*', '', dataset_string)    
    
    # Remove numbers
    dataset_string = re.sub(r'\d+', '', dataset_string)  
    
    # Remove punctuation
    #dataset_string = re.sub(r"[;':@#?!\&/$]+ *", ' ', dataset_string)
    
    # Remove that funny diamond
    dataset_string = re.sub(r"U+FFFD ", ' ', dataset_string)

    # Remove 'rt' denoting start of new message
    dataset_string = re.sub(r"rt ", ' ', dataset_string)

    # Print updated string
    dataset_string

Tweet Tone Analyzer

In [218]:
# Initialize Tone Analyser Client

# Create authentication object
authenticator = IAMAuthenticator(nlu_apikey)

# Create tone_analyzer instance
natural_language_understanding = NaturalLanguageUnderstandingV1 (
    version="2017-09-21",
    authenticator=authenticator
)

# Set the service endpoint
natural_language_understanding.set_service_url(nlu_url)

In [219]:
# Emotional Tone Analysis
response = natural_language_understanding.analyze(
    text=dataset_string,
    features=Features(emotion=EmotionOptions(targets=['coral', 'coral reef']))).get_result()

print(json.dumps(response, indent=2))

{
  "usage": {
    "text_units": 4,
    "text_characters": 33618,
    "features": 1
  },
  "language": "en",
  "emotion": {
    "targets": [
      {
        "text": "coral",
        "emotion": {
          "sadness": 0.130359,
          "joy": 0.455424,
          "fear": 0.106911,
          "disgust": 0.011906,
          "anger": 0.019639
        }
      },
      {
        "text": "coral reef",
        "emotion": {
          "sadness": 0.128174,
          "joy": 0.457788,
          "fear": 0.102401,
          "disgust": 0.010972,
          "anger": 0.019684
        }
      }
    ],
    "document": {
      "emotion": {
        "sadness": 0.154923,
        "joy": 0.396473,
        "fear": 0.103936,
        "disgust": 0.014794,
        "anger": 0.027123
      }
    }
  }
}


In [237]:
sia = SentimentIntensityAnalyzer()
sia.polarity_scores(dataset_string)

{'neg': 0.005, 'neu': 0.947, 'pos': 0.048, 'compound': 0.9983}